Importar las librerias

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.impute import SimpleImputer
import seaborn as sb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix, accuracy_score


Importar el data set


In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "test.csv" with length 28629 bytes
User uploaded file "train.csv" with length 61194 bytes


In [3]:
datatrain = pd.read_csv('train.csv', index_col='PassengerId')
datatest = pd.read_csv('test.csv')


df = datatrain.drop(['Name','Ticket'], axis = 'columns')
dt = datatest.drop(['Name','Ticket','PassengerId'], axis = 'columns')

df.columns = ['Superviviente', 'clase', 'genero', 'edad', 'SibSp', 'Parch', 
              'tarifa', 'cubierta', 'puerto']
dt.columns = ['clase', 'genero', 'edad', 'SibSp', 'Parch', 
              'tarifa', 'cubierta', 'puerto']

df.genero = df.genero.map({'female' : 'mujer',
                           'male' : 'hombre'}, na_action = None)
dt.genero = dt.genero.map({'female' : 'mujer',
                           'male' : 'hombre'}, na_action = None)


## codificar la variable cubierta - error


In [4]:
def substring_in_string (value, substrings):
    for substring in substrings:
        if substring in value:
            return substring

cubiertas = ['A', 'B', 'C', 'D', 'E', 'F', 'T', 'G', 'Unknown']

df.cubierta.fillna("Unknown", inplace=True)
dt.cubierta.fillna("Unknown", inplace=True)

df['cubierta'] = df ['cubierta'].map(lambda x: substring_in_string(x, cubiertas))
dt['cubierta'] = dt ['cubierta'].map(lambda x: substring_in_string(x, cubiertas))

## Codificar como factor la variable de clasificación


In [5]:
df.Superviviente = pd.Categorical(df.Superviviente)


## Convertir las variables categoricas a categorias


In [6]:
cual_col = ['clase', 'genero', 'SibSp', 'Parch', 'cubierta', 'puerto']
ordi_col = ['clase', 'SibSp', 'Parch']
cuanti_col = ['edad', 'tarifa']

for c in cual_col:
    if c in ordi_col:
        df[c] = pd.Categorical(df[c], ordered=True)
        dt[c] = pd.Categorical(dt[c], ordered=True)
    else:
        df[c] = pd.Categorical(df[c])
        dt[c] = pd.Categorical(dt[c])

df.clase = pd.Categorical(df.clase, ordered=True, categories=[3,2,1])
dt.clase = pd.Categorical(dt.clase, ordered=True, categories=[3,2,1])

df.Parch = pd.Categorical(df.Parch, ordered=True, categories=[0,1,2,3,4,5,6,9])


# PREPARAR LOS DATOS PARA EL MODELO


## Gestión de Variables

### Crear variable adulto

In [7]:
df.columns

df.loc[(df.edad <= 15), "adulto"] = "joven"
df.adulto.fillna("adulto", inplace=True)

#### En el conjunto de test


In [8]:
dt.loc[(dt.edad <= 15), "adulto"] = "joven"
dt.adulto.fillna("adulto", inplace=True)

In [9]:
df.adulto = pd.Categorical(df.adulto)
dt.adulto = pd.Categorical(dt.adulto)
cual_col.append("adulto")


## Detectar y eliminar outliers


### Detectar y eliminar outliers de la categoria cubierta T


In [10]:
df.drop(df[df.cubierta == "T"].index, axis=0, inplace=True)

df.cubierta = pd.Categorical(df.cubierta, categories=["A","B","C","D","E","F","G","Unknown"])

### Detectar y eliminar outliers de la categoria adulto


In [11]:
#### Calculo de estadisticos adultos
Q1_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.25)
Q3_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.75)
IQR_edad_adulto = Q3_edad_adulto-Q1_edad_adulto

#### Calculo de los bigotes adultos
BI_edad_adulto = Q1_edad_adulto - 1.5 * IQR_edad_adulto
BS_edad_adulto = Q3_edad_adulto + 1.5 * IQR_edad_adulto

#### Ubicación de outliers adultos
ubi_outliers_adulto = (df.edad[df.adulto == "adulto"] < BI_edad_adulto) | (df.edad[df.adulto == "adulto"] > BS_edad_adulto)
 
outliers_adulto = df[df.adulto == "adulto"][ubi_outliers_adulto]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto.index, axis=0, inplace=True)


### Detectar y eliminar outliers de la variable adultos, agrupada por genero:


In [12]:
#### Calculo de estadisticos adultos-hombres 
Q1_ed_ad_hombre = df.edad[df.adulto == "adulto"][df.genero == "hombre"].quantile(0.25)
Q3_ed_ad_hombre = df.edad[df.adulto == "adulto"][df.genero == "hombre"].quantile(0.75)
IQR_ed_ad_hombre = Q3_ed_ad_hombre-Q1_ed_ad_hombre

#### Calculo de los bigotes adultos-hombres
BI_ed_ad_hombre = Q1_ed_ad_hombre - 1.5 * IQR_ed_ad_hombre
BS_ed_ad_hombre = Q3_ed_ad_hombre + 1.5 * IQR_ed_ad_hombre

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_hombre = (df.edad[df.adulto == "adulto"][df.genero == "hombre"] < BI_ed_ad_hombre) | (df.edad[df.adulto == "adulto"][df.genero == "hombre"] > BS_ed_ad_hombre)
 
outliers_adulto_hombre = df[df.adulto == "adulto"][df.genero == "hombre"][ubi_outliers_adulto_hombre]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_hombre.index, axis=0, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


### Detectar y eliminar outliers de la variable adultos, agrupada por supervivientes:


In [13]:
#### Calculo de estadisticos adultos-Supervivientes 
Q1_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.25)
Q3_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.75)
IQR_ed_ad_super = Q3_ed_ad_super-Q1_ed_ad_super

#### Calculo de los bigotes adultos-Supervivientes

BI_ed_ad_super = Q1_ed_ad_super - 1.5 * IQR_ed_ad_super
BS_ed_ad_super = Q3_ed_ad_super + 1.5 * IQR_ed_ad_super

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_super = (df.edad[df.adulto == "adulto"][df.Superviviente == 1] < BI_ed_ad_super) | (df.edad[df.adulto == "adulto"][df.Superviviente == 1] > BS_ed_ad_super)

outliers_adulto_super = df[df.adulto == "adulto"][df.Superviviente == 1][ubi_outliers_adulto_super]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_super.index, axis=0, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


### SEGUNDA ITERACION eliminar outliers de la variable adultos, agrupada por supervivientes:


In [14]:
#### Calculo de estadisticos adultos-Supervivientes 
Q1_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.25)
Q3_ed_ad_super = df.edad[df.adulto == "adulto"][df.Superviviente == 1].quantile(0.75)
IQR_ed_ad_super = Q3_ed_ad_super-Q1_ed_ad_super

#### Calculo de los bigotes adultos-Supervivientes
BI_ed_ad_super = Q1_ed_ad_super - 1.5 * IQR_ed_ad_super
BS_ed_ad_super = Q3_ed_ad_super + 1.5 * IQR_ed_ad_super

#### Ubicación de outliers adultos-Supervivientes
ubi_outliers_adulto_super = (df.edad[df.adulto == "adulto"][df.Superviviente == 1] < BI_ed_ad_super) | (df.edad[df.adulto == "adulto"][df.Superviviente == 1] > BS_ed_ad_super)
 
outliers_adulto_super = df[df.adulto == "adulto"][df.Superviviente == 1][ubi_outliers_adulto_super]

#### Eliminación de outliers adultos-Supervivientes
df.drop(outliers_adulto_super.index, axis=0, inplace=True)


### Volver a calcular el Bigote superior de adulto para edad

Q1_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.25)
Q3_edad_adulto = df.edad[df.adulto == "adulto"].quantile(0.75)
IQR_edad_adulto = Q3_edad_adulto-Q1_edad_adulto
BI_edad_adulto = Q1_edad_adulto - 1.5 * IQR_edad_adulto
BS_edad_adulto = Q3_edad_adulto + 1.5 * IQR_edad_adulto


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  del sys.path[0]


## Tratamiento de los NA


### En el conjunto de entrenamiento


In [15]:
#### Completar los NA de hombres jovenes (masters) con la media

edad_master = datatrain.Age[datatrain.Name.str.contains("Master")]
ed_master_mean = edad_master.mean()

df.loc[(datatrain.Age.isna()) & (datatrain.Name.str.contains("Master")), "edad"] = np.ceil(ed_master_mean)


##### Los masters que quedan estaban codificados como adultos - OJO
df.loc[(df.edad <= 15) & (df.genero == "hombre"), "adulto"] = "joven"


#### Completar los NA de los adultos con la media

df.edad.fillna(df.edad.mean(), inplace=True)

### En el conjunto de test 


In [16]:
#### Hay un NAN en tarifa

dt.tarifa.fillna(dt.tarifa.mean(), inplace=True)



#### Los masters
edad_master_t = datatest.Age[datatest.Name.str.contains("Master")]
ed_master_mean_t = edad_master_t.mean()
#edad_master_t.max()

dt.loc[(datatest.Age.isna()) & (datatest.Name.str.contains("Master")), "edad"] = np.ceil(ed_master_mean_t)

##### Los masters que quedan, estaban codificados como adultos - OJO
dt.loc[(dt.edad <= 15) & (dt.genero == "hombre"), "adulto"] = "joven"


#### Completar los NA de los adultos con la media

dt.edad.fillna(dt.edad.mean(), inplace=True)

### Obtener la matriz de caracteristicas y la variable independiente


In [29]:
X = df.iloc[:, 1:]
y = df.iloc[:, 0]

x_t = dt.copy()

x0 = pd.concat([X,x_t], axis=0)


## Codificar variables categoricas


In [30]:
dummi_col = cual_col.copy()    

for x in ordi_col:
    dummi_col.remove(x)     
       
x0 = pd.get_dummies(x0, prefix=dummi_col, columns=dummi_col, drop_first=True)

## Dividir el conjunto de datos en entrenamiento y test


In [31]:
X = x0.iloc[0:len(df),:]
x_t = x0.iloc[len(df):,:]


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=0, stratify=y)


## Escalado de valores

In [33]:
sc_X = StandardScaler()
sc_X_t = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
x_t = sc_X_t.fit_transform(x_t)

# Seleccionar los modelos


## Aplicar el modelo de Deep Learning RNA



### Inicializar la RNA


In [34]:
classi_rna = Sequential()


### Añadir las capas de entrada y la primera capa oculta


In [35]:
classi_rna.add(Dense(units = 9, kernel_initializer = "uniform",  
                     activation = "relu", input_dim = 16))

### Añadir la segunda capa oculta


In [36]:
classi_rna.add(Dense(units = 9, kernel_initializer = "uniform",  activation = "relu"))


### Añadir la tercera capa oculta


In [37]:
classi_rna.add(Dense(units = 9, kernel_initializer = "uniform",  activation = "relu"))


### Añadir la capa de salida


In [38]:
classi_rna.add(Dense(units = 1, kernel_initializer = "uniform",  activation = "sigmoid"))


### Compilar la RNA


In [39]:
classi_rna.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])


### Ajustamos la RNA al Conjunto de Entrenamiento


In [40]:
classi_rna.fit(X_train, y_train,  batch_size = 10, epochs = 100)


Epoch 1/100
70/70 [==============================] - 14s 2ms/step - loss: 0.6926 - accuracy: 0.5712
Epoch 2/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6840 - accuracy: 0.6564
Epoch 3/100
70/70 [==============================] - 0s 2ms/step - loss: 0.6284 - accuracy: 0.7608
Epoch 4/100
70/70 [==============================] - 0s 2ms/step - loss: 0.5181 - accuracy: 0.7762
Epoch 5/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4470 - accuracy: 0.7985
Epoch 6/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4494 - accuracy: 0.8147
Epoch 7/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4455 - accuracy: 0.8074
Epoch 8/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4364 - accuracy: 0.8152
Epoch 9/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4494 - accuracy: 0.8005
Epoch 10/100
70/70 [==============================] - 0s 2ms/step - loss: 0.4215 - accuracy: 0.8120
Epoch 11

## Evaluar el modelo y calcular predicciones finales


### Predicción de los resultados con el Conjunto de Testing


In [41]:
y_pred_rna  = classi_rna.predict(X_test)
y_pred_rna = (y_pred_rna>0.5)

### Elaborar una matriz de confusión


In [42]:
cmRNA = confusion_matrix(y_test, y_pred_rna)
accuracy_score(y_test, y_pred_rna)



0.8742857142857143

# CREAR EL OUTPUT


## Output RNA


In [44]:


y_out_rna = classi_rna.predict(x_t)
y_out_rna = (y_out_rna>0.5)
y_out_rna = pd.DataFrame(y_out_rna, columns=["Survived"])
y_out_rna = y_out_rna.astype(int)

outRNA = pd.concat([datatest.PassengerId, y_out_rna], axis=1)
outRNA.set_index('PassengerId', inplace=True)

#outRNA.to_csv('subm_colab_RNA_100821.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#with open('subm_am_colab_xgb_100821.csv', 'w') as f:
#  f.write(outXGB)

#files.download('subm_am_colab_xgb_100821.csv')